In [ ]:
%cd /content/drive/MyDrive/M.Tech CS_2022-23/Project/One Class Learning

/content/drive/MyDrive/M.Tech CS_2022-23/Project/One Class Learning


In [ ]:
import os
import pickle
import numpy as np
import torch
from collections import defaultdict
from tqdm import tqdm
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cuda = True if device == 'cuda' else False
if cuda: torch.cuda.set_device(0)

##Data Modules

In [ ]:
def instance_normalize(data, num_channels):
  data = np.array(data).astype(np.float32)
  data = data / 255.0  
  data = data / (np.linalg.norm(data, axis=1, keepdims=True))
  data = data - np.expand_dims(np.mean(data, 1), 1) + 0.006
  data = data.reshape((data.shape[0], num_channels, -1))
  return data

In [ ]:
data_path = './data/CIFAR10/'

train_dataset = torchvision.datasets.CIFAR10(data_path, train=True, download=True)
test_dataset  = torchvision.datasets.CIFAR10(data_path, train=False, download=True)
train_dataset.data = instance_normalize(train_dataset.data.reshape(train_dataset.data.shape[0], -1), num_channels=3)
test_dataset.data = instance_normalize(test_dataset.data.reshape(test_dataset.data.shape[0], -1), num_channels=3)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [ ]:
train_transform = transforms.ToTensor()
test_transform = transforms.ToTensor()

In [ ]:
def get_class_c(x, y, c):    
  y = np.array(y)
  pos_c = np.argwhere(y == c)
  pos_c = list(pos_c[:, 0])
  class_c_data = [x[i] for i in pos_c]
  return class_c_data, [c]*len(pos_c)


class DatasetMaker(Dataset):
  def __init__(self, true_label, data, targets, transform_func=None):
    super().__init__()    
    self.data = data
    self.targets = targets
    self.true_label = true_label
    self.transform_func = transform_func

  def __getitem__(self, idx):
    img, target = self.data[idx], self.targets[idx]
    if self.transform_func:
      img = self.transform_func(img)
    return img, target == self.true_label

  def __len__(self):
    return len(self.targets)


def prepare_oc_dataset(dataset, class_label):
  # get single class (class_label) data
  data, targets = get_class_c(dataset.data, dataset.targets, class_label)
  return data, targets

In [ ]:
batch_size = 256

##Model

In [ ]:
class MLP(nn.Module):
  def __init__(self, num_outputs, num_hiddens):
    super(MLP, self).__init__()
    self.net = nn.Sequential(nn.Flatten(),
                             *[nn.Sequential(nn.LazyLinear(nh), nn.LeakyReLU()) for nh in num_hiddens],
                             nn.LazyLinear(num_outputs))    
  
  def forward(self, X):
    return self.net(X)

In [ ]:
model = MLP(num_outputs=1, num_hiddens=[900, 300])
model.to(device);

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


##Training

In [ ]:
def train_epoch(model, dataloader, optimizer):
  model.train()  
  train_loss = []
  
  for inputs, _ in dataloader:
    inputs = inputs.to(device)
    inputs.requires_grad = True
    features = model(inputs)
    
    nll = -torch.log(torch.sigmoid(features) + 1e-2).mean()
    h_reg = compute_gradient_penalty(inputs, features).mean()
    loss = nll + 0.1 * h_reg
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    train_loss.append(loss.detach().cpu())

  return train_loss

In [ ]:
def compute_gradient_penalty(inputs, features):  
  gradients = autograd.grad(outputs=features,
                            inputs=inputs,
                            grad_outputs=torch.ones_like(features).to(device),
                            create_graph=True,
                            retain_graph=True)[0]
  
  return (gradients.norm(2, dim=1)) ** 12

##Testing

In [ ]:
def test(model, dataloader):
  model.eval()
  outputs, labels = [], []
  for input_batch, label_batch in dataloader:
    input_batch = input_batch.to(device)    
    outputs.append(torch.sigmoid(model(input_batch)).detach().cpu())
    labels.append(label_batch)    
  return torch.cat(outputs), torch.cat(labels)

# def compute_accuracy(outputs, labels, averaged=True):
#   preds = (outputs >= 0.95).type(labels.dtype)
#   compare = (preds == labels.reshape(-1)).type(torch.float32)
#   return compare.mean() if averaged else compare

def compute_auc(labels, outputs):
  return roc_auc_score(labels, outputs)

In [ ]:
num_epochs = 50

log = {}
log['train_loss'] = []

results = defaultdict(list)
min_outputs = defaultdict(lambda:float('inf'))
max_outputs = defaultdict(lambda:0)
  

# For each class
for true_label in range(10):
  
  # prepare the data and
  train_data, train_targets = prepare_oc_dataset(train_dataset, true_label)
  train_oc_dataset = DatasetMaker(true_label, train_data, train_targets, train_transform)
  test_oc_dataset = DatasetMaker(true_label, test_dataset.data, test_dataset.targets, test_transform)
  train_loader = DataLoader(train_oc_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_oc_dataset, batch_size=batch_size, shuffle=False)
      
  # instantiate the model
  model = MLP(num_outputs=1, num_hiddens=[900, 300]).to(device)
  # optimizer
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

  # train the model    
  for epoch in range(num_epochs):
    batchwise_losses = train_epoch(model, train_loader, optimizer)
    train_epoch_loss = np.mean(batchwise_losses)
    log['train_loss'].append(train_epoch_loss)

  # test time
  model_outputs, labels = test(model, test_loader)

  # compute the results
  auc = compute_auc(labels, model_outputs)
  
  # store the results
  results[true_label].append(auc)
  min_outputs[true_label] = min(min_outputs[true_label], torch.min(model_outputs))
  max_outputs[true_label] = max(max_outputs[true_label], torch.max(model_outputs))

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functional

##Mean of the experiments

In [ ]:
for label, scores in results.items():
  print(f'Class: {label}\tMean AUC Score: {np.mean(scores):.4f}')

Class: 0	Mean AUC Score: 0.5842
Class: 1	Mean AUC Score: 0.4765
Class: 2	Mean AUC Score: 0.4835
Class: 3	Mean AUC Score: 0.4689
Class: 4	Mean AUC Score: 0.4540
Class: 5	Mean AUC Score: 0.5108
Class: 6	Mean AUC Score: 0.4238
Class: 7	Mean AUC Score: 0.5325
Class: 8	Mean AUC Score: 0.6006
Class: 9	Mean AUC Score: 0.5686
